In [1]:
import requests
import pandas as pd
from google_currency import convert
import json

In [2]:
# as max num of rows is always 2000 this parameters should be the same 
# (Except for key words, obv)

number_of_pages = 20
per_page = 100
key_words = ["Нарративный дизайнер", "Редактор", "Копирайтер"]

In [3]:
# convert to pandas df

def convert_to_df(data):
    df_page_list = pd.DataFrame(data)
    df = pd.DataFrame()

    for item in df_page_list['items']:
        df_item = pd.DataFrame(item)
        df = pd.concat([df, df_item]).reset_index(drop=True)
    
    for column in df.columns:
        try:
            keys = df[column][0].keys()
            for key in keys:
                new_col_name = column+'_'+key
                df[new_col_name] = [d.get(key) for d in df[column]]
        except:
            df[column] = df[column]

    print('Total row count: ', len(df))
    return df

In [4]:
# get ex rates

def get_ex_rates(df, cur_column='salary_currency'):
    ex_rates = []
    for cur in df[cur_column].unique():
        ex_rate = float(json.loads(convert(cur.lower(), 'rur', 1))['amount'])
        ex_rates.append([cur, ex_rate])
    ex_rates = pd.DataFrame(ex_rates)
    ex_rates.columns = [cur_column, 'ex_rate']
    df = df.merge(ex_rates, how='left', on = cur_column)
    return df

In [5]:
# get taxes

def get_taxes(df, code_col='salary_currency'):

# sorry it was faster to make it as a list 
# please feel free to add more countries

    taxes = pd.DataFrame({
        code_col:['RUR', 'EUR', 'BYR', 'USD', 'UZS', 'KZT', 'GEL', 'KGS'],
        'tax_rate':[0.13, 0.378, 0.13, 0.37, 0.12, 0.10, 0.2, 0.15]})
    df = df.merge(taxes, how='left', on = code_col)
    return df

In [6]:
# calculate aprx salary (in rubles, avg between from and to)

def calculate_aprx_salary(df):
    df['aprx_salary_rub'] = df[['salary_from', 'salary_to']].mean(axis=1)
    df['aprx_salary_rub'] = df['aprx_salary_rub']*df['ex_rate']
    df.loc[df['salary_gross']==True, 'aprx_salary_rub'] = \
    df['aprx_salary_rub'] - df['aprx_salary_rub']*df['tax_rate']
    df['aprx_salary_q'] = round(df['aprx_salary_rub']/25000,0)*25000
    
    return df

In [7]:
# load general data with parameters (hh limit - 2000 rows)

data=[]
for word in key_words:
    for i in range(number_of_pages):
        url = 'https://api.hh.ru/vacancies'
        par = {'text': word, 
               'per_page':per_page, 
               'page':i,
               'only_with_salary': 'true',
               'employment': 'full',
              }
        r = requests.get(url, params=par)
        e=r.json()
        data.append(e) 

total_vacancies = e['found']

df_general = convert_to_df(data)
df_general = get_ex_rates(df_general)
df_general = get_taxes(df_general)
df_general = calculate_aprx_salary(df_general)

Total row count:  3495


In [8]:
print('Total vacancies: ', total_vacancies)
print('Mean salary (LIM: 2000 vacancies): ', df_general['aprx_salary_rub'].mean())
df_general.groupby('aprx_salary_q').count()[['id']]

Total vacancies:  1494
Mean salary (LIM: 2000 vacancies):  62556.02163919885


,id
aprx_salary_q,
0.0,12
25000.0,725
50000.0,1495
75000.0,670
100000.0,345
125000.0,116
150000.0,59
175000.0,28
200000.0,17


In [9]:
# load data with salary>125000 rub (hh limit - 2000 rows)

data=[]
for word in key_words:
    for i in range(number_of_pages):
        url = 'https://api.hh.ru/vacancies'
        par = {'text': word, 
               'per_page':per_page, 
               'page':i,
               'only_with_salary': 'true',
               'employment': 'full',
               'salary':125000
              }
        r = requests.get(url, params=par)
        e=r.json()
        data.append(e) 
        
selected_vacancies = e['found']

df_selected = convert_to_df(data)
df_selected = get_ex_rates(df_selected)
df_selected = get_taxes(df_selected)
df_selected = calculate_aprx_salary(df_selected)

Total row count:  746


In [10]:
print('Total vacancies: ', total_vacancies)
print('selected_vacancies: ', selected_vacancies)
print('Share of jobs with 125k+ salary: ', 
      round(selected_vacancies/total_vacancies,2)*100,
      '%')

Total vacancies:  1494
selected_vacancies:  232
Share of jobs with 125k+ salary:  16.0 %


In [11]:
from collections import Counter

df_selected["snippet_requirement"] = df_selected["snippet_requirement"].fillna('')
for i in ['<highlighttext>', '</highlighttext>', '<highlighttext>', '</highlighttext>']:
    df_selected["snippet_requirement"] = df_selected["snippet_requirement"].replace(i, '')
    
    
words = Counter(" ".join(df_selected["snippet_requirement"]).split()).most_common(100)

In [12]:
words

[('и', 566),
 ('в', 471),
 ('работы', 360),
 ('Опыт', 330),
 ('с', 271),
 ('Знание', 174),
 ('-', 155),
 ('опыт', 152),
 ('от', 136),
 ('Умение', 134),
 ('или', 104),
 ('графическими', 104),
 ('на', 100),
 ('работать', 99),
 ('Владение', 95),
 ('знание', 74),
 ('графических', 73),
 ('лет.', 69),
 ('пользователь', 62),
 ('для', 61),
 ('программами', 60),
 ('MS', 58),
 ('к', 55),
 ('Excel,', 55),
 ('умение', 52),
 ('–', 51),
 ('графические', 50),
 ('владение', 49),
 ('Microsoft', 47),
 ('Навыки', 45),
 ('<highlighttext>редакторы</highlighttext>', 40),
 ('ПК', 40),
 ('не', 38),
 ('программ', 38),
 ('по', 37),
 ('Высшее', 37),
 ('Adobe', 37),
 ('области', 36),
 ('Уверенное', 36),
 ('Понимание', 35),
 ('<highlighttext>редакторами</highlighttext>.', 35),
 ('3', 33),
 ('лет', 33),
 ('сфере', 33),
 ('Уверенный', 33),
 ('навыки', 32),
 ('3-х', 32),
 ('<highlighttext>редакторами</highlighttext>', 31),
 ('основ', 30),
 ('/', 29),
 ('уровень', 29),
 ('Office', 28),
 ('уровне', 28),
 ('документации

In [13]:
df_general.to_csv('GeneralHHoutput.csv')
df_selected.to_csv('RichHHoutput.csv')